# Interaction approximations across orbital configurations

This notebook compares the exact Legendre-series evaluation of the vector resonant relaxation interaction with its asymptotic approximations (with and without low-order corrections) for representative orbital configurations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from vrr_Omegas import (
    Orbit,
    OrbitPair,
    ExactSeriesEvaluator,
    AsymptoticEvaluator,
    AsymptoticWithCorrectionsEvaluator,
)

plt.style.use('seaborn-v0_8-colorblind')


In [ ]:
# Define representative orbital configurations
M_CENTRAL = 1.0
G_CONST = 1.0

configurations = {
    "Circular/Circular": {
        "primary": Orbit(a=1.0, e=0.0, m=1.0),
        "secondary": Orbit(a=2.5, e=0.0, m=1.5),
    },
    "Eccentric (non-overlap)": {
        "primary": Orbit(a=1.0, e=0.2, m=1.1),
        "secondary": Orbit(a=2.5, e=0.4, m=1.3),
    },
    "Eccentric (overlap)": {
        "primary": Orbit(a=1.0, e=0.6, m=1.0),
        "secondary": Orbit(a=1.8, e=0.5, m=1.4),
    },
    "Mixed (circ/ecc)": {
        "primary": Orbit(a=1.0, e=0.0, m=1.0),
        "secondary": Orbit(a=2.0, e=0.5, m=1.2),
    },
}

cos_grid = np.linspace(-0.95, 0.95, 160)

exact = ExactSeriesEvaluator(ell_max=30)
asymptotic = AsymptoticEvaluator()
hybrid = AsymptoticWithCorrectionsEvaluator(lmax_correction=6)

evaluators = {
    "Exact": exact,
    "Asymptotic": asymptotic,
    "Asymp. + corr.": hybrid,
}

results = {}
for label, cfg in configurations.items():
    data = {name: {"hamiltonian": [], "omega": []} for name in evaluators}
    for cos_inc in cos_grid:
        pair = OrbitPair(
            cfg["primary"],
            cfg["secondary"],
            cos_inc,
            G=G_CONST,
            M_central=M_CENTRAL,
        )
        for name, evaluator in evaluators.items():
            res = evaluator.evaluate_pair(pair)
            data[name]["hamiltonian"].append(res.hamiltonian)
            data[name]["omega"].append(res.omega)
    for name in evaluators:
        data[name]["hamiltonian"] = np.array(data[name]["hamiltonian"])
        data[name]["omega"] = np.array(data[name]["omega"])
    results[label] = data


In [ ]:
# Plot Hamiltonian and precession frequency comparisons
fig, axes = plt.subplots(2, len(configurations), figsize=(4 * len(configurations), 6), sharex='col')

method_styles = {
    "Exact": dict(color='black', linewidth=2.0),
    "Asymptotic": dict(color='tab:blue', linestyle='--'),
    "Asymp. + corr.": dict(color='tab:orange', linestyle='-.'),
}

for col, (config_label, data) in enumerate(results.items()):
    ax_H = axes[0, col]
    ax_O = axes[1, col]

    for method_name, series in data.items():
        style = method_styles.get(method_name, {})
        ax_H.plot(cos_grid, series["hamiltonian"], label=method_name, **style)
        ax_O.plot(cos_grid, series["omega"], label=method_name, **style)

    ax_H.set_title(config_label)
    ax_H.set_ylabel('Hamiltonian')
    ax_O.set_ylabel(r'$\Omega$')
    ax_O.set_xlabel(r'$\cos i$')
    ax_H.grid(alpha=0.2)
    ax_O.grid(alpha=0.2)

    if col > 0:
        ax_H.set_ylabel('')
        ax_O.set_ylabel('')

handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=len(evaluators), bbox_to_anchor=(0.5, 1.02))
fig.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()
